In [11]:
import sympy
from sympy import Symbol, sympify, Sum, oo, ln, log, sin, cos, pi, factorial, E, Wild, pretty, S, solve_univariate_inequality, Interval

def analysiere_reihenkonvergenz_final(term_str: str, variable_str: str = 'k', start_wert: int = 1):
    """
    Analysiert eine unendliche Reihe mit einer Hierarchie von Konvergenztests,
    inklusive eines direkten Vergleichstests.
    (Version 6 - Finale Version)
    """
    resultat = {
        "reihe": f"Σ({term_str}) von {variable_str}={start_wert} bis ∞",
        "konvergenz": "Nicht bestimmt",
        "test": "Keiner der implementierten Tests war erfolgreich."
    }
    
    try:
        var = Symbol(variable_str, integer=True, positive=True)
        erlaubte_funktionen = {'n': var, 'k': var, 'pi': pi, 'E': E, 'ln': ln, 'log': log, 'sin': sin, 'cos': cos, 'factorial': factorial}
        term_expr = sympify(term_str, locals=erlaubte_funktionen)

        # --- Test 1: Nullfolgenkriterium ---
        grenzwert_term = limit(term_expr, var, oo)
        if grenzwert_term != 0:
            resultat["konvergenz"] = "Divergent"
            resultat["test"] = "Nullfolgenkriterium (Terme gehen nicht gegen 0)"
            return resultat

        # --- Test 2: p-Reihe ---
        p = Wild('p')
        match = term_expr.match(1/var**p)
        if match and match.get(p) is not None and match[p].is_Number:
            if match[p] > 1:
                resultat["konvergenz"] = "Konvergent"
                resultat["test"] = f"p-Reihe mit p={match[p]} > 1"
                return resultat
            else:
                resultat["konvergenz"] = "Divergent"
                resultat["test"] = f"p-Reihe mit p={match[p]} <= 1"
                return resultat

        # --- Test 3: Absolute Konvergenz durch Vergleich ---
        if term_expr.has(sin(var)) or term_expr.has(cos(var)):
            vergleichs_term = term_expr.subs([(sin(var), 1), (cos(var), 1)])
            if analysiere_reihenkonvergenz_final(str(abs(vergleichs_term)), variable_str, start_wert)['konvergenz'] == 'Konvergent':
                resultat["konvergenz"] = "Konvergent (Absolut)"
                resultat["test"] = f"Vergleichskriterium (mit Σ|{vergleichs_term}|)"
                return resultat

        # --- Test 4: Direkter Vergleich mit harmonischer Reihe ---
        try:
            # Versuche zu beweisen, dass unser Term größer ist als 1/k
            vergleichs_term = 1/var
            ungleichung = term_expr > vergleichs_term
            # Löse die Ungleichung für k
            intervall = solve_univariate_inequality(ungleichung, var, relational=False)
            
            # Gilt die Ungleichung für alle k ab einem bestimmten Punkt bis unendlich?
            if intervall.end == oo and intervall.start.is_finite:
                resultat["konvergenz"] = "Divergent"
                resultat["test"] = f"Direktes Vergleichskriterium (Minorante: {vergleichs_term})"
                return resultat
        except Exception:
            # Wenn das Lösen der Ungleichung fehlschlägt, machen wir einfach weiter
            pass

        # --- Fallback: Allgemeine SymPy-Funktion ---
        reihe = Sum(term_expr, (var, start_wert, oo))
        konvergenz = reihe.is_convergent()
        if konvergenz is True:
            resultat["konvergenz"] = "Konvergent"; resultat["test"] = "Allgemeiner Test von SymPy"
        elif konvergenz is False:
            resultat["konvergenz"] = "Divergent"; resultat["test"] = "Allgemeiner Test von SymPy"
            
    except Exception as e:
        resultat["konvergenz"] = "Fehler"; resultat["test"] = f"Fehler bei der Analyse: {e}"

    return resultat

# --- Hauptprogramm ---
if __name__ == "__main__":
    test_reihen = [
        "ln(k)/k",
        "1/k**2",
        "sin(k) / k**3"
    ]
    for reihe_term in test_reihen:
        analyse = analysiere_reihenkonvergenz_final(reihe_term, 'k', 1)
        print("─" * 60)
        for key, value in analyse.items():
            print(f"  {key.capitalize()}: {value}")
    print("─" * 60)

────────────────────────────────────────────────────────────
  Reihe: Σ(ln(k)/k) von k=1 bis ∞
  Konvergenz: Divergent
  Test: Direktes Vergleichskriterium (Minorante: 1/k)
────────────────────────────────────────────────────────────
  Reihe: Σ(1/k**2) von k=1 bis ∞
  Konvergenz: Konvergent
  Test: p-Reihe mit p=2 > 1
────────────────────────────────────────────────────────────
  Reihe: Σ(sin(k) / k**3) von k=1 bis ∞
  Konvergenz: Konvergent (Absolut)
  Test: Vergleichskriterium (mit Σ|k**(-3)|)
────────────────────────────────────────────────────────────
